pyannote-video (GITHUB)
---
**A toolkit for face detection, tracking, and clustering in videos**

In [ ]:
!apt update
!apt upgrade -y
!uname -m && cat /etc/*release
!gcc --version
!uname -r

In [ ]:
!nvidia-smi

To access Google Drive Folder and Files

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/GoogleColab"

Create Directory & Copy File From Google Drive

In [ ]:
%mkdir pyannote-data
!cp -r "/content/drive/My Drive/GoogleColab/BRICS.mp4" pyannote-data/

**Upgrading all packages with pip**

In [ ]:
!pip list --outdated --format=freeze | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U

Then, install pyannote-video and its dependencies:

In [ ]:
!pip install imageio==2.4.1 imgaug==0.2.5 mido==1.2.6 python-rtmidi==1.2
!pip install pyannote-video

Finally, download sample video and dlib models:

In [ ]:
!git clone https://github.com/pyannote/pyannote-data.git
!git clone https://github.com/davisking/dlib-models.git
!bunzip2 dlib-models/dlib_face_recognition_resnet_model_v1.dat.bz2
!bunzip2 dlib-models/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
ls pyannote-data/

In [ ]:
!git clone https://github.com/pyannote/pyannote-video.git

In [ ]:
%pylab inline

Shot segmentation
---

In [ ]:
!pyannote-structure.py --help

In [ ]:
!pyannote-structure.py shot --verbose pyannote-data/BRICS.mp4 \
                                      pyannote-data/BRICS.shots.json

Detected shot boundaries can be visualized using pyannote.core notebook support:

In [ ]:
from pyannote.core.json import load_from
shots = load_from('pyannote-data/BRICS.shots.json')
shots

Face processing
---

In [ ]:
!pyannote-face.py --help

In [ ]:
!pyannote-face.py track --verbose --every=0.5 pyannote-data/BRICS.mp4 \
                                              pyannote-data/BRICS.shots.json \
                                              pyannote-data/BRICS.track.txt

**Face tracking**

Face tracks can be visualized using demo mode:

In [ ]:
!pyannote-face.py demo pyannote-data/BRICS.mp4 \
                       pyannote-data/BRICS.track.txt \
                       pyannote-data/BRICS.track.mp4

In [ ]:
import io
import base64
from IPython.display import HTML
video = io.open('pyannote-data/BRICS.track.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii')))

Facial landmarks and face embedding
---

In [ ]:
!pyannote-face.py extract --verbose pyannote-data/BRICS.mp4 \
                                    pyannote-data/BRICS.track.txt \
                                    dlib-models/shape_predictor_68_face_landmarks.dat \
                                    dlib-models/dlib_face_recognition_resnet_model_v1.dat \
                                    pyannote-data/BRICS.landmarks.txt \
                                    pyannote-data/BRICS.embedding.txt

Face clustering
---
Once embeddings are extracted, let's apply face track hierarchical agglomerative clustering.
The distance between two clusters is defined as the average euclidean distance between all embeddings.

In [ ]:
from pyannote.video.face.clustering import FaceClustering
clustering = FaceClustering(threshold=0.6)

In [ ]:
face_tracks, embeddings = clustering.model.preprocess('pyannote-data/BRICS.embedding.txt')
face_tracks.get_timeline()

In [ ]:
face_tracks

In [ ]:
result = clustering(face_tracks, features=embeddings)


In [ ]:
result

In [ ]:
from pyannote.core import notebook, Segment
notebook.reset()
notebook.crop = Segment(0, 42)
#mapping = {8: 'Vladimir Putin', 22: 'Michel Temer', 23:'Cyril Ramaphosa', 40:'Xi Jinping', 39:'Narendra Modi', 6:'False Negative'}
mapping = {16: 'Vladimir_Putin', 26: 'Michel_Temer', 30:'Cyril_Ramaphosa', 37:'Xi_Jinping', 33:'Narendra_Modi', 22:'False_Negative'}
result = result.rename_labels(mapping=mapping)
result

In [ ]:
with open('pyannote-data/BRICS1.labels.txt', 'w') as fp:
    for _, track_id, cluster in result.itertracks(yield_label=True):
        fp.write(f'{track_id} {cluster}\n')

In [ ]:
!pyannote-face.py demo pyannote-data/BRICS.mp4 \
                       pyannote-data/BRICS.track.txt \
                       --label=pyannote-data/BRICS1.labels.txt \
                       pyannote-data/BRICS1.final.mp4

In [ ]:
import io
import base64
from IPython.display import HTML
video = io.open('pyannote-data/BRICS1.final.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii')))

In [ ]:
from google.colab import files
files.download('pyannote-data/BRICS1.final.mp4')

In [ ]:
ls pyannote-data/

In [ ]:
 !cp -r pyannote-data/BRICS* "/content/drive/My Drive/GColab_ML_DL/pyannote_data/"

In [ ]:
ls pyannote-data/BRICS*